# Pythonディスクリプタの理解と応用

Pythonのディスクリプタは、言語仕様(Python2.7では 3.4.2.2, 3.4.2.3)で説明されているように、通常の属性値検索を迂回する特殊な機能をもたらす。さらに、この“属性値検索の迂回”に加えて、“属性値の取得”の際にディスクリプタのメソッドが呼び出されるという動的な性質から、計算型の属性値の実現に用いられている。本報告書では、Pythonの2つの特徴(機能)について、自前でのその機能のサブセットをユーザ定義ディスクリプタとして実装するこで、ディスクリプタへの理解を深め、さらに Python 言語にはない、関数のオーバロードを実装することで、ディスクリプタのもつ機動性の高さに触れる。

なお、この文書では Python 2.7 を想定する。Python 3.x ではディスクリプタのAPIが若干異なるが基本思想は変わらない。

## ディスクリプタのおさらい

言語仕様ではクラスがディスクリプタになるため要件について次のように記述されている。

> 以下のメソッドは、このメソッドを持つクラス (いわゆる デスクリプタ(descriptor) クラス) のインスタンスが、 オーナー (owner) クラスに存在するときにのみ適用されます (デスクリプタは、オーナーのクラス辞書か、その親のいずれかのクラス辞書になければなりません)。 以下の例では、"属性" とは、名前がオーナークラスの `__dict__` のプロパティ (porperty) のキーであるような属性を指します。
> 
> object.\_\_get\_\_(self, instance, owner)
>
>     オーナクラスの属性を取得する (クラス属性へのアクセス) 際や、オーナクラスのインスタンスの属性を取得する (インスタンス属性へのアクセス) 場合に呼び出されます。 owner は常にオーナクラスです。一方、 instance は属性へのアクセスを仲介するインスタンスか属性が owner を介してアクセスされる場合は None になります。このメソッドは (計算された) 属性値を返すか、 AttributeError 例外を送出しなければなりません。
>
> object.\_\_set\_\_(self, instance, value)
> 
>     オーナクラスのインスタンス instance 上の属性を新たな値 value に設定する際に呼び出されます。
>
> object.\_\_delete\_\_(self, instance)
> 
>     オーナクラスのインスタンス instance 上の属性を削除する際に呼び出されます。

また

> デスクリプタが `__set__()` と `__delete__()` またはそのどちらかを定義していれば、データデスクリプタとなります; もし両方とも定義しなければ、非データデスクリプタです。通常、データデスクリプタでは、 `__get__()` と `__set__()` を定義し、一方、非データデスクリプタには `__get__()` メソッドしかありません。

と記載されているように、3つのメソッド全てを実装する必要はないが、実装しているメソッドの種類によってディスクリプタは二種(データと非データ)に分類され、

> データデスクリプタは、インスタンス辞書内で属性値が再定義されても、常にこの値をオーバライドします。対照的に、非データデスクリプタの場合には、属性値はインスタンス側でオーバライドされます。

とあるようように、それらは属性値の設定に関して挙動が異なる。

### 簡単な例

まず、簡単な例でディスクリプタの挙動を確認する。最初は非データディスクリプタの定義である。

In [32]:
class NonDataDescriptor(object):
    def __get__(self, instance, owner):
        print 'NonDataDescriptor.__get__: instance:', instance, " owner:", owner

つづいてデータディスクリプタも定義する。

In [33]:
class DataDescriptor(object):
    def __get__(self, instance, owner):
        print 'DataDescriptor.__get__: instance:', instance, " owner:", owner
    def __set__(self, instance, value):
        print 'DataDescriptor.__set__: instance:', instance, " value:", value

ディスクリプタは単独で存在する意義はなく、ディスクリプタになる要件の冒頭に

> 以下のメソッドは、このメソッドを持つクラス (いわゆる デスクリプタ(descriptor) クラス) のインスタンスが、 オーナー (owner) クラスに存在するときにのみ適用されます (デスクリプタは、オーナーのクラス辞書か、その親のいずれかのクラス辞書になければなりません)。

とあるように、属性値アクセスをカスタマイズしたい目的のクラスに、クラス辞書に含まれる形式で取り込む必要がある。

クラス `Test` に属性 `a` と `b` を持たせたいとすると、例えば、
```python
class Test(object):
    def __init__(self):
        self.a = None           # 初期値
        self.b = None           # 初期値
```
のように書くことだろう(サブクラスでも挙動を保持するなら `__new__` を使うが)。 
この場合、`Test`のインスタンスの属性値である `a` や `b` は属性値を再設定しなければ固定的であり、一方任意に書き換えたり、削除すらも可能である。この挙動を変更するのにディスクリプタが用いられる。

先程の要件の冒頭部分は、この `Test` を次のように書き換えることを意味している。

In [34]:
class Test(object):
    a = NonDataDescriptor()
    b = DataDescriptor()

インスタンスの属性値であるはずの `a` や `b` をクラスの辞書に登録している。一応、確認してみる。

In [35]:
Test.__dict__

dict_proxy({'__dict__': <attribute '__dict__' of 'Test' objects>,
            '__doc__': None,
            '__module__': '__main__',
            '__weakref__': <attribute '__weakref__' of 'Test' objects>,
            'a': <__main__.NonDataDescriptor at 0x401a860>,
            'b': <__main__.DataDescriptor at 0x401a278>})

実際に `Test` のインスタンスを作成して属性 `a` にアクセスする。

In [36]:
t = Test()
t.a

NonDataDescriptor.__get__: instance: <__main__.Test object at 0x0000000004108EF0>  owner: <class '__main__.Test'>


同様に `b` にアクセスする。

In [37]:
t.b

DataDescriptor.__get__: instance: <__main__.Test object at 0x0000000004108EF0>  owner: <class '__main__.Test'>


どちらも、`NonDataDescriptor`, `DataDescriptor` の `__get__` メソッドが呼び出されたことが確認できた。

### 非データディスクリプタとデータディスクリプタとの違い
ここで一応、非データディスクリプタとデータディスクリプタとの違いについても確認しておく。
再掲すると
> 非データデスクリプタの場合には、属性値はインスタンス側でオーバライドされます。

そこで、インスタンス `t` の属性 `a` に値を設定してみる。

In [38]:
t.__dict__['a'] = 1
print 't.__dict__:', t.__dict__
t.a

t.__dict__: {'a': 1}


1

先程と異なり `t.a` による属性値のアクセスで `NonDataDescriptor.__get__` が呼び出されなくなった。

一方で、データディスクリプタに対して同様にすると、

In [39]:
t.__dict__['b'] = 1
print 't.__dict__:', t.__dict__
t.b

t.__dict__: {'a': 1, 'b': 1}
DataDescriptor.__get__: instance: <__main__.Test object at 0x0000000004108EF0>  owner: <class '__main__.Test'>


> データデスクリプタは、インスタンス辞書内で属性値が再定義されても、常にこの値をオーバライドします。

とあるように、インスタンス辞書内の属性値を無視して `DataDescriptor.__get__` が呼び出された。なお、この例で、`a` や `b` の属性値の設定に `t.a = 1` としなかったのは、特に `t.b = 1` の場合には `DataDescriptor.__set__` が呼ばれてインスタンス `t` の辞書を書き換えることができないためである。

### \_\_get\_\_ の複雑な挙動

ディスクリプタに必要なメソッドのうち `__get__` だけが異様に複雑な仕様となっている。

> `object.__get__(self, instance, owner)`
>       オーナクラスの属性を取得する (クラス属性へのアクセス) 際や、オーナクラスのインスタンスの属性を取得する (インスタンス属性へのアクセス) 場合に呼び出されます。 owner は常にオーナクラスです。一方、 instance は属性へのアクセスを仲介するインスタンスか属性が owner を介してアクセスされる場合は None になります。このメソッドは (計算された) 属性値を返すか、 AttributeError 例外を送出しなければなりません。

ここで言うオーナークラスは、先程の例ではクラス `Test` を意味する。`__get__` メソッドはクラス属性へのアクセスの場合でも、インスタンス属性へのアクセスの場合でも呼び出され、それぞれに応じて `instance` 引数の値が異なる。

In [40]:
t = Test()
t.a
Test.a

NonDataDescriptor.__get__: instance: <__main__.Test object at 0x0000000003F2E128>  owner: <class '__main__.Test'>
NonDataDescriptor.__get__: instance: None  owner: <class '__main__.Test'>


このあと、この挙動の違いが Python の特徴である、結合メソッドオブジェクト・非結合メソッドオブジェクトへ繋がることを確認する。

## 関数とメソッドオブジェクト

まず単純な例から、関数といわゆるメソッド(クラス属性としての関数)、メソッドオブジェクトとの関係を確認する。

In [10]:
def function(self, *args):
    print 'function: self:', self, ' args:', args
    
class Test(object):
    method = function

上の例では、一つのメソッドをもつクラスを定義している。これは、ほぼ、

```python
class Test(object):
    def method(self, *args):
        print 'function: self:', self, ' args:', args
```

と等価であるが、関数定義を `class` 定義の外に出すことで、Pythonでは、他のオブジェクト指向言語でいうところのメソッドと関数に違いがないことを示している。

Python において特徴的なのは、メソッド(関数)定義において、メソッドの操作対象となるオブジェクト(他言語ではレシーバと呼ばれたりする)を第一引数で明示的に受けとるということである。通常、その引数は `self` と書かれるが、レシーバがクラスオブジェクトとなる場合は `cls` とか `klass` と書かれることもある。いずれにしろ、引数の名称は言語仕様では規定されない。

さて、メソッドの定義においては、レシーバとなるオブジェクトを引数で明示的に受け取るように記述するのであるが、インスタンスオブジェクト `obj` のメソッド `method` を呼び出す場合の構文、`obj.method(...)` では `obj` が暗黙のうちに渡される。一方、`obj` の所属するクラス `C` に対しても `C.method(...)` の形式でメソッド呼び出しができるが、この場合は `method` の第１引数に、`C` (または`C`のサブクラス)のインスタンスを明示的に渡さなければならない。

以下、それぞれの例を見る。

In [11]:
t = Test()
print t
t.method('1st argument')

function: self: <__main__.Test object at 0x000000000401A630>  args: ('1st argument',)


`method` の第１引数 `self` にレシーバである `t` 自身が渡されていることが確認できる。同じ構文で、`Test.method` を呼び出すと、

In [12]:
Test.method('1st argument')

TypeError: unbound method function() must be called with Test instance as first argument (got str instance instead)

`method` の第１引数が '1st argeument' という文字列であり、`Test` のインスタンスでないためエラーとなる。では、`t` を明示的に渡してみる。

In [13]:
Test.method(t, '1st argument')

function: self: <__main__.Test object at 0x000000000401A630>  args: ('1st argument',)


ちなみに、これは、*ほぼ* `function(t, '1st argument')` と等価である。

In [14]:
function(t, '1st argument')

function: self: <__main__.Test object at 0x000000000401A630>  args: ('1st argument',)


*ほぼ* と書いたのは、`function` の直接呼び出しでは、第１引数が `Test` のインスタンスであることをチェックできないためであり、`Test.method` は `function` とは違うものだからである。

In [15]:
function('1st argument')

function: self: 1st argument  args: ()


`function` (これは、`Test.__dict__['function']` と同じ) と `t.method`、そして `Test.method` との違いを、もう少し掘り下げる。それぞれの型(所属するクラス)と文字列表現を見てみる。

In [16]:
print type(function)
print function
print type(t.method)
print t.method
print type(Test.method)
print Test.method

<type 'function'>
<function function at 0x0000000002394908>
<type 'instancemethod'>
<bound method Test.function of <__main__.Test object at 0x000000000401A630>>
<type 'instancemethod'>
<unbound method Test.function>


type()の表示により、`t.method` と `Test.method` は双方とも `instancemethod` と呼ばれるクラスのインスタンス(オブジェクト)となっていることがわかる。いわゆるメソッドオブジェクトだ。type()を通さない文字列表現では、前者は `bound method`(結合メソッド)となっていて、後者は `unbound method`(非結合メソッド)となっている。同じメソッドオブジェクトでも何かが違うということだが、この違いが、`method` 呼び出し時の暗黙の引数の有無に関係していることは想像に難くない。

実際 `instancemethod` は `im_self`, `im_class`, `im_func` という属性をもっており、これを調べると、それはすぐに判明する。

In [17]:
print t.method.im_self
print t.method.im_class
print t.method.im_func

<class '__main__.Test'>
<function function at 0x0000000002394908>


In [18]:
print Test.method.im_self
print Test.method.im_class
print Test.method.im_func

None
<class '__main__.Test'>
<function function at 0x0000000002394908>


`t.method` の方は `im_self` が自身の `t` をもっている。一方 `Test.method` の方は `im_self` が `None` であり、これらの違いがレシーバオブジェクトの暗黙の引き渡しの違いとしてあらわれている。

以上の事柄は、言語仕様 3.2.標準型の階層の一部で厳密に記述されている。

> ユーザ定義メソッド (user-defined method)
>
>    ... 中略 ...
>
>    クラスの属性を (おそらくクラスのインスタンスを介して) 取得する際には、その属性がユーザ定義の関数オブジェクト、非結合 (unbound) のユーザ定義メソッドオブジェクト、あるいはクラスメソッドオブジェクトであれば、ユーザ定義メソッドオブジェクトが生成されることがあります。属性がユーザ定義メソッドオブジェクトの場合、属性を取得する対象のオブジェクトが属するクラスがもとのメソッドオブジェクトが定義されているクラスと同じクラスであるか、またはそのサブクラスであれば、新たなメソッドオブジェクトだけが生成されます。それ以外の場合には、もとのメソッドオブジェクトがそのまま使われます。
>
>    クラスからユーザ定義関数オブジェクトを取得する方法でユーザ定義メソッドオブジェクトを生成すると、 im_self 属性は None になり、メソッドオブジェクトは非結合 (unbound) であるといいます。クラスのインスタンスからユーザ定義関数オブジェクトを取得する方法でユーザ定義メソッドオブジェクトを生成すると、 im_self 属性はインスタンスになり、メソッドオブジェクトは結合 (bound) であるといいます。どちらの場合も、新たなメソッドの im_class 属性は、メソッドの取得が行われたクラスになり、 im_func 属性はもとの関数オブジェクトになります。
>
>    クラスやインスタンスから他のユーザ定義メソッドオブジェクトを取得する方法でユーザ定義メソッドオブジェクトを生成した場合、その動作は関数オブジェクトの場合と同様ですが、新たなインスタンスの im_func 属性はもとのメソッドオブジェクトの属性ではなく、新たなインスタンスの属性になります。
>
>    クラスやインスタンスからクラスメソッドオブジェクトを取得する方法でユーザ定義メソッドオブジェクトを生成した場合、 im_self 属性はクラス自体となり、 im_func 属性はクラスメソッドの根底にある関数オブジェクトになります。
>
>    非結合ユーザ定義メソッドオブジェクトの呼び出しの際には、根底にある関数 (im_func) が呼び出されます。このとき、最初の引数は適切なクラス (im_class) またはサブクラスのインスタンスでなければならないという制限が課されています。
>
>    結合ユーザ定義メソッドオブジェクトの呼び出しの際には、根底にある関数 (im_func) が呼び出されます。このとき、クラスインスタンス (im_self) が引数の先頭に挿入されます。例えば、関数 f() の定義が入ったクラスを C とし、 x を C のインスタンスとすると、 x.f(1) の呼び出しは C.f(x, 1) と同じになります。
>
>    ユーザ定義メソッドオブジェクトがクラスオブジェクトから派生した際、 im_self に記憶されている "クラスインスタンス" はクラス自体になります。これは、 x.f(1) や C.f(1) の呼び出しが根底にある関数を f としたときの呼び出し f(C,1) と等価になるようにするためです。


このように記述すればかなり量になるメソッドオブジェクトの仕様であるが、これらを、「関数オブジェクトを非データディスクリプタとすること」でシンプルに実現している。次節では、関数、インスタンスメソッドの代替となるユーザ定義クラスを構築して、その挙動を再現する。

## 私家版の関数オブジェクトとメソッドオブジェクト

`instancemethod` と同じような動作をするクラス `MyInstanceMethod` を定義する。これは比較的容易である。

- コンストラクタの引数は `instancemethod` に合わせ、根底となる関数、インスタンス、そのインスタンスの所属クラスとする。
- `im_self`, `im_class`, `im_func` を模倣する。
- `instancemethod` 同様にcallableとする。つまり、`__call__` を実装する。
- `__call__` では `im_self` により暗黙の第１引数を渡すかどうかを決定して根底となる関数を呼び出す。

In [19]:
class MyInstanceMethod(object):
    def __init__(self, base_func, instance, owner):
        self.im_func = base_func
        self.im_self = instance
        self.im_class = owner

    def __call__(self, *args, **kws):
        if self.im_self is None:
            if not args or not isinstance(args[0], self.im_class):
                raise TypeError('[MY] unbound method require instance of %s as first argument' %
                                self.im_class.__name__)
            return self.im_func(*args, **kws)
        else:
            return self.im_func(self.im_self, *args, **kws)

    def __repr__(self):
        if self.im_self is None:
            return '[MY] unbound method (%s.%s)' % (
                self.im_class.__name__, self.im_func.__name__)
        else:
            return '[MY] bound method (%s.%s of %s)' % (
                self.im_class.__name__, self.im_func.__name__, str(self.im_self))

    __str__ = __repr__

さて、次は肝となる私家版関数のクラスである。関数オブジェクトはバイトコードを含むcodeオブジェクトを包むラッパーのような存在である(省略時引数やクロージャ変数は関数オブジェクトに含まれる)。ただ、codeオブジェクトを知っていても、それを引数つきで実行する手段が Python インタープリタ外部には公開されていないため、私家版関数の内部では本来の関数オブジェクトを使用する。ここでは、私家版関数クラスが非データディスクリプタとして定義されることが重要なので、それ以外には目を瞑る。

In [20]:
class MyFunction(object):
    def __init__(self, original_func):
        self.__func = original_func

    def __getattr__(self, attr):
        return getattr(self.__func, attr)

    def __call__(self, *args, **kws):
        print '[MyFunction]', self.__func.__name__, args, kws
        return self.__func(*args, **kws)

    def __get__(self, instance, owner):
        return MyInstanceMethod(self, instance, owner)

`MyFunction` の肝は `__get__` メソッドであり、このメソッドの引数がそのまま `MyInstanceMethodType` に渡っている。
<BR>
それでは、`Test` クラスを再定義して挙動を確かめる。

In [21]:
# def myfunctoin(): ... で MyFunction オブジェクトを生成するこはできないので、
# 既存の function から MyFunction オブジェクトを生成する。
myfunction = MyFunction(function)

class Test(object):
    method = myfunction

t = Test()

In [22]:
t.method('1st arguement')

[MyFunction] function (<__main__.Test object at 0x00000000040FB748>, '1st arguement') {}
function: self: <__main__.Test object at 0x00000000040FB748>  args: ('1st arguement',)


`t.method` に対して `MyFunction.__get__(myfunction, t, Test)` が呼ばれるため、`MyInstanceMethod` が(私家版)結合メソッドオブジェクトを生成し、暗黙の第１引数をともなったメソッド呼び出しができている。

In [23]:
Test.method('1st argument')

TypeError: [MY] unbound method require instance of Test as first argument

`instancemethod` の場合と同様に、第１引数の型をチェックしている。

次のように(私家版)非結合メソッドの呼び出しで `t` を明示的に渡せば仕様通りの振舞いをみせる。

In [24]:
Test.method(t, '1st argument')

[MyFunction] function (<__main__.Test object at 0x00000000040FB748>, '1st argument') {}
function: self: <__main__.Test object at 0x00000000040FB748>  args: ('1st argument',)


### classmethod、staticmethod

関数デコレータである `classmethod`, `staticmethod` も非データディスクリプタである。ここでは `classmethod` の私家版 `MyClassMethod` を定義しよう。`classmethod` は `instancemethod` と違って callable ではないので、私家版でも `__call__` を実装しない。

In [26]:
class MyClassMethod(object):
    def __init__(self, f):
        self.__func = f

    def __get__(self, instance, owner):
        return MyInstanceMethod(self.__func, owner, type(owner))

関数クラスの `__get__` との違いは、クラスメソッドの方は `instance` 引数を無視して、`owner` と `owner` の所属(メタクラスで普通は `type`)を固定的にインスタンスメソッドのコンストラクタに渡すことである。

In [27]:
class Test(object):
    @MyClassMethod
    def cmethod(cls, *args):
        print 'cmethod: cls:', cls, ' args:', args
        
t = Test()

In [28]:
t.cmethod('1st argument')
Test.cmethod('1st argument')

cmethod: cls: <class '__main__.Test'>  args: ('1st argument',)
cmethod: cls: <class '__main__.Test'>  args: ('1st argument',)


`staticmethod` の方は押して知るべしであろう。

## 私家版 property の実装

組み込みオブジェクト `property` は計算型の属性を容易にクラスに導入する優れもので、ある属性名 `attr` に対する `getter` と `setter` を次のような構文で導入することができる。

```python
class Test(object):
    def __init__(self):
        self.__internal_attr = 0
       
    @property
    def attr(self):
        return self.__internal_attr
    
    @attr.setter
    def attr(self, value):
        self.__internal_attr = value * 1000
```

この定義により、`attr` は `Test` のデータディスクリプタとして導入される。この `property` もディスクリプタである。そこで、`getter`と`setter`を定義できる私家版プロパティ `MyProperty`を定義してみる。

まず、`getter` の導入のみを考慮する。

- コンストラクタがデコレータの要件を満たするように、getter となる関数を引数にとる。
- ディスクリプタとしての `__get__` を持ち、コンストラクタに指定された getter を呼び出す。

In [29]:
class MyProperty(object):
    def __init__(self, getter):
        self.__getter = getter

    def __get__(self, instance, owner):
        if instance is None:
            return self         # instance が None の場合のこの動作は property に合わせたもの
        return self.__getter(instance)

簡単な例で動作を確認してみる。

In [31]:
class Test(object):
    def __init__(self):
        self.__value = 'hello'
        
    @MyProperty
    def attr(self):
        return self.__value

t = Test()
t.attr

'hello'

次に `setter` の導入であるが、`@MyProperty` による `getter` の導入により、上の例の `attr` は既に `MyProperty` のインスタンスとなっていることに注意する。`setter`導入時の

```python
    @attr.setter
    def attr(self, value):
        ...

```

という構文は、`MyProperty` が `setter` という名前のメソッドを持っていて、それが `setter`関数のデコレータとなっているということを意味する。まとめると、`MyProperty` は以下のようになる。

In [66]:
class MyProperty(object):
    def __init__(self, getter):
        self.__getter = getter

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.__getter(instance)
    
    def setter(self, setter):
        self.__setter = setter
        return self

    def __set__(self, instance, value):
        return self.__setter(instance, value)

`MyProperty` の `setter` が自身(`self`)を戻すところがミソである。

In [67]:
class Test(object):
    def __init__(self):
        self.__value = 0
        
    @MyProperty
    def attr(self):
        self.__value += 1
        return self.__value
    
    @attr.setter
    def attr(self, value):
        self.__value = value

t = Test()
print t.attr
print t.attr
t.attr = 100
print t.attr
print t.attr

1
2
101
102


## 関数オーバーロードの実装

最後に、`instancemethod` や `property` の私家版の実装から得た知見をもとに、Python言語にはない、関数のオーバーロード機能をディスクリプタを用いて実装してみる。

なお、C++などの静的型付け言語での関数オーバロードは、シグネチャ(全ての引数の型)をベースにコンパイル時に解決しているが、Python は動的型付けなので実行時の解決となる。また、関数定義からは引数の型は不明なので、ここでの実装は以下の方針をとる。

- オーバロードしたい関数はデコレータを用いて第１引数の型情報を与える。
- オーバロードしたい関数が引数をとらない場合は型情報は与えない。
- オーバロードの解決は実引数の第１引数の型のみを用いて行う。
- 引数なしの関数呼び出しは、型情報のない関数を呼び出す。

オーバロードを実現するクラスは `Overload` とし、`property` と似たような以下のような記述を用いるものとする。

```python
@Overload()
def func():
  print 'func()'
  
@func.overload(int)
def func(*args):
  print 'func(int):', args
  
@func.overload(str)
def func(*args):
  print 'func(str):', args
```

### トップレベル関数に対する Overload (失敗)

手始めにトップレベル関数に対して `Overload` を構築してみる。`Overload` のコンストラクタは省略可能な引数 `arg1type` をとり、このデフォルト値を `type(None)` とする。このデフォルト値は、引数なしを意味する。`Overload` クラスでは、第１引数の型と実際のコーラブルオブジェクトとの対応を辞書で管理する。以下のコードが浮ぶであろう。

```python
class Overload(object):
    def __new__(cls, arg1type=type(None)):
        self = super(Overload, cls).__new__(cls)
        self.__funcs = {}
        ...
```

問題はここからだ。通常の `__new__` は生成したクラスオブジェクトを返却する。この例でも、そのようにするとどうなるか？ 使用例の最初をみると、デコレータの定義から `def func(): ...` の後に `func = (Overload())(func)` が実行されるので以下の仕様が課される。

- `Overload` クラスは callable でなければならず `__call__` を持つ。
- `__call__` はデコレータとして使われるため、引数を一つだけとり、その引数にはデコレータ機能より `def func(): ...` の `func` が渡される。
- `__call__` ではコンストラクタに渡された arg1type の値と `__call__` に渡された関数を `self.__funcs`辞書へ登録する。
- `__call__` の結果は `func` に代入され、その後 `@func.overload()` というデコレータとして働かなければならないため、自信を返却する。

ここまでを満すコード片は以下のようになる。

```python
class Overload(object):
    def __new__(cls, arg1type=type(None)):
        self = super(Overload, cls).__new__(cls)
        self.__arg1type = arg1type
        self.__funcs = {}
        return self

    def __call__(self, func):
        self.__funcs[self.__arg1type] = func
        return self
```

ところがこの方法では、オーバロード関数の呼び出しができない。先の例では最初のデコレータ使用後に `func()` を呼び出すと、`Overload.__call__` が呼び出されるため、デコレート対象の本来の関数の呼び出しに到達しない。

### トップレベル関数に対する Overload

`Overload` のコンストラクタの結果が再度デコレータとして呼び出されるために、`__call__` の役割が被ってしまった。`__call__` はオーバーロード関数の呼び出しのためにとっておかなければならない。そこで一旦、コンストラクタとデコレーションを分離してしまって、常に `overload` メソッド(の結果)をデコレータとして使用するとしよう。それは、引数をもつデコレータの一般的な慣習に従う。

```python
class Overload(object):
    ...
    def overload(self, arg1type=type(None)):
        def _overload(f):
            self.__funcs[arg1type] = f
            return self
        return _overload
```

`overload` メソッドはデコレータとして呼ばれることになる `_overload` を内部にもち、これを返却する。返却された`_overload` はデコレータ機能から呼び出され、デコレート対象の関数が仮引数 `f` に渡される。`_overload` はクロージャ変数である `arg1type` と `f` の組合せを `self.__funcs` へ登録し、`Overload`インスタンスを返却する。この `Overload` インスタンスはデコレータ機能により再度 `func` へ設定される。

この方法であれば、`__call__` をオーバーロード関数の呼び出しとして確保しておくことができる。そして、`__new__` であるが、自身の中で生成した `self` を戻すのでなく、`overload` メソッドの呼び出し結果を戻せば良いのである。これに、`__call__` の実装をまとめると次のようになる。

In [41]:
class Overload(object):
    def __new__(cls, arg1type=type(None)):
        self = super(Overload, cls).__new__(cls)
        self.__funcs = {}
        return self.overload(arg1type)

    def overload(self, arg1type=type(None)):
        def _overload(f):
            self.__funcs[arg1type] = f
            return self
        return _overload

    def __call__(self, *args, **kwargs):
        if args:
            return self.__funcs[type(args[0])](*args, **kwargs)
        else:
            return self.__funcs[type(None)](**kwargs)

では試してみよう。

In [42]:
@Overload()
def func():
    print 'func()'

@func.overload(int)
def func(*args):
    print 'func(int):', args

@func.overload(str)
def func(*args):
    print 'func(str):', args

func()
func(10)
func('string')

func()
func(int): (10,)
func(str): ('string',)


同一の `func` という名前の呼び出しで、引数の型に応じて別々の関数が呼ばれていることが確認できた。

### インスタンスメソッドへの適用

先程定義して `Overload` はインスタンスメソッドについても動作するだろうか。簡単な例で試してみる。

In [49]:
class Test(object):
    @Overload()
    def method(self):
        print 'method(): self: ', self

t = Test()
t.method()

TypeError: method() takes exactly 1 argument (0 given)

案の定エラーとなった。レシーバオブジェクトが第１引数に挿入されないためだ。この理由は簡単で、`Test` の辞書にある `method` は `Overload` のインスタンスであって、`Overload` はディスクリプタでないからだ。このため、`t.method` という属性参照は `Test.__dict__['method']` そのものである、そのオブジェクトに対して空引数での呼び出しを行っているのである。

ということは、`Overload` に `__get__` を装備すれば良いということになる。では、`__get__` では何をすれば良いだろうか？ 上の例で考えると `__get__` の返却値に対して空引数での呼び出しが行われるので、`__get__` について以下の要件が課される。

- `__get__` は callable オブジェクト を返さねばならない。それを以下、`CO` と記述する。
- `CO` に本来の引数が渡るため、そこでオーバーロードの解決をしなければならない。
- もし、`t.method` でなく `Test.method` として呼ばれた場合は、`__get__` の `instance` は None であり、`CO` の最初の引数には `Test` のインスタンスが渡されるので、この場合は第2引数をオーバーロード解決に使わなければならない。
- 逆に `t.method` で呼ばれた場合は、オーバーロード解決後の関数を呼び出す場合にレシーバオブジェクトを引数の先頭に挿入しなければならない。

以上を考慮して、`__get__` は内部関数 `__call_method` を定義して、その関数を返却するだけとして、`__call_method` で上の要件の2番目以降を実装する。`__call_method` はおおむね、以下のようになる。

```python
    def __get__(self, instance, owner):
        def __call_method(*args, **kwargs):
            if instance is not None:
                args = (instance,) + args
            arg1 = args[1] if len(args) > 1 else None
            method = self.__funcs[type(arg1)]
            return method(*args, **kwargs)
        return __call_method
```

これに、`instance` が `None` の場合に `args[0]` が `onwer` のインスタンスであることのチェックを追加する。

In [55]:
class Overload(object):
    def __new__(cls, arg1type=type(None)):
        self = super(Overload, cls).__new__(cls)
        self.__funcs = {}
        return self.overload(arg1type)

    def overload(self, arg1type=type(None)):
        def _overload(f):
            self.__funcs[arg1type] = f
            return self
        return _overload

    def __call__(self, *args, **kwargs):
        if args:
            return self.__funcs[type(args[0])](*args, **kwargs)
        else:
            return self.__funcs[type(None)](**kwargs)

    def __get__(self, instance, owner):
        def __call_method(*args, **kwargs):
            if instance is None:
                if not args or not isinstance(args[0], owner):
                    raise TypeError('unbound method require instance of %s as first argument' % owner.__name__)
            else:
                args = (instance,) + args
            arg1 = args[1] if len(args) > 1 else None
            method = self.__funcs[type(arg1)]
            return method(*args, **kwargs)
        return __call_method

class Over(object):
    @Overload()
    def method(*args):
        print 'method():', args

    @method.overload(int)
    def method(*args):
        print 'method(int):', args

    @method.overload(str)
    def method(*args):
        print 'method(str):', args
        
ov = Over()
ov.method()
ov.method(123)
ov.method('method')
Over.method(ov)
Over.method(ov, 123)
Over.method(ov, 'method')

method(): (<__main__.Over object at 0x0000000004131048>,)
method(int): (<__main__.Over object at 0x0000000004131048>, 123)
method(str): (<__main__.Over object at 0x0000000004131048>, 'method')
method(): (<__main__.Over object at 0x0000000004131048>,)
method(int): (<__main__.Over object at 0x0000000004131048>, 123)
method(str): (<__main__.Over object at 0x0000000004131048>, 'method')


### クラスメソッドやスタティックメソッドへの対応

さて、これで完璧に思えるが、クラスメソッドなどにも通用するだろうか？

In [58]:
class Over(object):
    @Overload()
    @classmethod
    def cmethod(*args):
        print 'cmethod():', args

ov = Over()
ov.cmethod()

TypeError: 'classmethod' object is not callable

そう、`classmethod` オブジェクトは `callable` でないし、そもそも `Overload` はインスタンスメソッドの挙動だけを意識しているため動作しない。`__call_method` は `instance` が `None` でない場合に `instance` を args に挿入しているが、クラスメソッドの場合は `owner` を `args` に挿入しなければならない。

さて解決策であるが、そもそも `self.__funcs{}` には、関数、クラスメソッド、スタティックメソッドのいずれかが登録されており、それらは全てディスクリプタであるから、そのディスクリプタの `__get__` を直接呼び出せば良い。そして、`__call_method` はオーバーロードの解決にフォーカスする。最後の課題となるのが、`args` の何番目の引数でオーバーロードを解決するかだ。

- `instance` が None でなければ、`args` の第１引数はレシーバオブジェクトではないのでこれを用いる。
- `instance` が None の場合、オーバーロードの対象がインスタンスメソッドなら `args[1]` を用い、その他の場合は `args[0]` を用いる。

現状、オーバーロードの対象がインスタンスメソッドになるのかそうでないか(クラスメソッド、スタティックメソッド)かは不明なので、`overload` メソッドで登録する際に `_overload` の引数 `f` の素性(`type(f)`)を `self.__ftype` に登録すこととする。そうすれば、`self.__ftype` が `types.FunctionType` であればインスタンスメソッドが対象だと判断できる。これらを考慮すると 新しい `__call_method` は次のようになる。

```python
    def __get__(self, instance, owner):
        def __call_method(*args, **kwargs):
            if instance is None and self.__ftype is types.FunctionType:
                arg1 = args[1] if len(args) > 1 else None
            else:
                arg1 = args[0] if args else None
            method = self.__funcs[type(arg1)]
            return method.__get__(instance, owner)(*args, **kwargs)
        return __call_method
```

`self.__ftype` への登録や、登録時のチェックなどを含めて Overload を再構成する。

In [1]:
import types

class Overload(object):
    def __new__(cls, arg1type=type(None)):
        self = super(Overload, cls).__new__(cls)
        self.__funcs = {}
        self.__ftype = None;
        return self.overload(arg1type)

    def overload(self, arg1type=type(None)):
        def _overload(f):
            if self.__ftype is None:
                self.__ftype = type(f)
            elif self.__ftype is not type(f):
                raise TypeError('Method type mismatch')
            if arg1type in self.__funcs:
                raise TypeError('Duplicate overload for type: %s' % type(arg1type))
            self.__funcs[arg1type] = f
            return self
        return _overload

    def __call__(self, *args, **kwargs):
        if args:
            return self.__funcs[type(args[0])](*args, **kwargs)
        else:
            return self.__funcs[type(None)](**kwargs)

    def __get__(self, instance, owner):
        def __call_method(*args, **kwargs):
            if instance is None and self.__ftype is types.FunctionType:
                arg1 = args[1] if len(args) > 1 else None
            else:
                arg1 = args[0] if args else None
            method = self.__funcs[type(arg1)]
            return method.__get__(instance, owner)(*args, **kwargs)
        return __call_method

トップレベルの関数を含めて総合的に動作を確認する。

In [2]:
@Overload()
def func():
    print 'func()'

@func.overload(int)
def func(*args):
    print 'func(int):', args

@func.overload(str)
def func(*args):
    print 'func(str):', args

func()
func(10)
func('string')

class Over(object):
    @Overload()
    def method(*args):
        print 'method():', args

    @method.overload(int)
    def method(*args):
        print 'method(int):', args

    @method.overload(str)
    def method(*args):
        print 'method(str):', args

    @Overload()
    @classmethod
    def cmethod(*args):
        print 'cmethod():', args

    @cmethod.overload(int)
    @classmethod
    def cmethod(*args):
        print 'cmethod(int):', args

    @cmethod.overload(str)
    @classmethod
    def cmethod(*args):
        print 'cmethod(str):', args

    @Overload()
    @staticmethod
    def smethod(*args):
        print 'smethod():', args

    @smethod.overload(int)
    @staticmethod
    def smethod(*args):
        print 'smethod(int):', args

    @smethod.overload(str)
    @staticmethod
    def smethod(*args):
        print 'smethod(str):', args
        
ov = Over()
ov.method()
ov.method(123)
ov.method('method')
Over.method(ov)
Over.method(ov, 123)
Over.method(ov, 'method')

ov.cmethod()
ov.cmethod(123)
ov.cmethod('method')
Over.cmethod()
Over.cmethod(123)
Over.cmethod('method')

ov.smethod()
ov.smethod(123)
ov.smethod('method')
Over.smethod()
Over.smethod(123)
Over.smethod('method')

func()
func(int): (10,)
func(str): ('string',)
method(): (<__main__.Over object at 0x0000000003F38B00>,)
method(int): (<__main__.Over object at 0x0000000003F38B00>, 123)
method(str): (<__main__.Over object at 0x0000000003F38B00>, 'method')
method(): (<__main__.Over object at 0x0000000003F38B00>,)
method(int): (<__main__.Over object at 0x0000000003F38B00>, 123)
method(str): (<__main__.Over object at 0x0000000003F38B00>, 'method')
cmethod(): (<class '__main__.Over'>,)
cmethod(int): (<class '__main__.Over'>, 123)
cmethod(str): (<class '__main__.Over'>, 'method')
cmethod(): (<class '__main__.Over'>,)
cmethod(int): (<class '__main__.Over'>, 123)
cmethod(str): (<class '__main__.Over'>, 'method')
smethod(): ()
smethod(int): (123,)
smethod(str): ('method',)
smethod(): ()
smethod(int): (123,)
smethod(str): ('method',)


### 一般のcallableオブジェクトへ

この最新の `Overload` では、クラス上でのオーバロードに関して、callableオブジェクトがディスクリプタであることを前提にしていたが、クラスのメンバーが一般の callable オブジェクトである可能性もある。このケースまで対応するなら、`__call_method` の最後の部分で、`method` が `__get__` を持っていれば `__get__` 経由で、そうでなければ `method` を直接呼び出すようにすれば良い。

## 最後に

これまで、ディスクリプタとデコレータの組み合わせで、Pythonの中核の機能が実現され、さらに関数オーバロードのような新たな言語要素が追加できるのを見た。

メソッドオブジェクトの生成やレシーバオブジェクトの引数への挿入など他のプログラミング言語であれば、(コンパイル時あるいはインタープリタで)言語処理系として直接処理されるものであろうし、プロパティなども `getter` や `setter` などの専用の構文を持つだろう。

Pythonでは、言語使用全般から見れば非常に小さいAPI仕様のディスクリプタと、これまた非常に単純な糖衣構文であるデコレータとを組み合わせて、大きなしかも互いに無関係な言語要素を実現している。この絶妙な基本要素の選択と、それを用いた言語仕様の構築が Python 言語の醍醐味であり、その設計思想に触れるのは日常的にPythonを使用しないエンジニアにとっても有意義であると考える。